# Last Baseline script of Big Contest

Last Baseline script. Hope this helps ours.

In [1]:
import numpy as np
import pandas as pd

# Load Data

In [2]:
train = pd.read_pickle("../all_df.p")
train.head(3)

,날짜,순위,영화명,개봉일,매출액,매출액 점유율,매출액증감 (전일대비),매출액증감율 (전일대비),누적매출액,관객수,...,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
0,2016년 07월 06일(수),1,봉이 김선달,2016-07-06 00:00:00,1022864200,0.332,1018335200,224.8,1127087200,136622,...,747,3391,한국,한국,"(주)엠픽처스,(주)에스엔케이픽처스",씨제이이앤엠(주),12세이상관람가,사극,박대민,"유승호,조재현,고창석,라미란,시우민,연우진,전석호,서예지,주영호"
1,2016년 07월 06일(수),2,굿바이 싱글,2016-06-29 00:00:00,695297000,0.225,-276778500,-0.3,9864818015,91390,...,650,2757,한국,한국,"호두앤유 엔터테인먼트(주),(주)영화사람",(주)쇼박스,15세이상관람가,"코미디,드라마",김태곤,"김혜수,마동석,김현수,김용건,서현진,곽시양,전석호,이미도,이성민,황미영"
2,2016년 07월 06일(수),3,도리를 찾아서,2016-07-06 00:00:00,617720600,0.2,614570600,195.1,641605600,76938,...,728,3446,미국,미국,NaN,월트디즈니컴퍼니코리아(주),전체관람가,"애니메이션,어드벤처,코미디",앤드류 스탠튼,"엘렌 드제너스,알버트 브룩스,다이안 키튼"


In [3]:
test = pd.read_pickle("../test.p")
test

,날짜,영화명,개봉일,대표국적,국적,제작사,배급사,등급,장르,감독,배우
0,2016년 09월 30일(금),"고산자, 대동여지도",2016-09-07,한국,한국,(주)시네마서비스,씨제이이앤엠(주),전체관람가,사극,강우석,"차승원,유준상,김인권,남지현,신동미,남경읍,태인호,공형진,김종수"
1,2016년 09월 30일(금),매그니피센트 7,2016-09-14,미국,미국,,유니버설픽쳐스인터내셔널 코리아(유),15세이상관람가,"액션, 서부극(웨스턴)",안톤 후쿠아,"덴젤 워싱턴,크리스 프랫,이병헌,에단 호크,맷 보머,빈센트 도노프리오,해일리 베넷,..."
2,2016년 09월 30일(금),카페 소사이어티,2016-09-14,미국,미국,,CGV아트하우스,15세이상관람가,"멜로/로맨스, 드라마",우디 앨런,"제시 아이젠버그, 크리스틴 스튜어트, 스티브 카렐, 블레이크 라이블리"


# Feature Engineering

- 최종 누적관객수 15만 이하 노이즈 제거
- 상영기간 변수 추가, 상영기간 [0 - 30] 제외한 데이터 제거
- 연휴포함 변수 추가 (설날, 추석이 포함되어 있는지 유무)
- 영화정보 변수 추가 (대표국적, 제작사, 배급사, 등급, 장르, 감독, 배우)

In [4]:
import re
import datetime

In [5]:
name = train[train["누적관객수"] >= 150000]['영화명']
name = name.unique()
train = train.loc[train['영화명'].isin(name)]
train.shape

(17598, 23)

## During Feature
- 상영기간 : 날짜 - 개봉일
- 상영기간 [0 - 30] 제외한 데이터 제거

In [6]:
train["날짜"] = train["날짜"].apply(lambda date: re.sub('[(년월화수목금토일)]', '', date))
train["날짜"] = train["날짜"].apply(lambda date: date.replace(" ", "-"))

train["개봉일"] = pd.to_datetime(train["개봉일"], errors='coerce')

train["상영기간"] = pd.to_datetime(train["날짜"]) - pd.to_datetime(train["개봉일"])
train = train.dropna(subset=['상영기간'])

train = train[train["상영기간"] >= '0 days']
train = train[train["상영기간"] < '30 days']

train["상영기간"] = train["상영기간"].astype('timedelta64[D]').astype(int)

## Holidays Feature
- 14년 추석 : 09/07 ~ 09/10
- 15년 설날 : 02/18 ~ 02/22
- 15년 추석 : 09/26 ~ 09/29
- 16년 설날 : 02/07 ~ 02/10

In [7]:
holidays = {'14_sul': {'day_start': '2014-01-30', 'day_end': '2014-02-02'},
            '14_chu': {'day_start': '2014-09-07', 'day_end': '2014-09-10'}, 
            '15_sul': {'day_start': '2015-02-18', 'day_end': '2015-02-22'},
            '15_chu': {'day_start': '2015-09-26', 'day_end': '2015-09-29'},
            '16_sul': {'day_start': '2016-02-07', 'day_end': '2016-02-10'}
           }

In [8]:
train["연휴포함"] = ((holidays['14_chu'].get('day_start') > train["개봉일"]) 
                 & (holidays['14_chu'].get('day_end') < train["날짜"]))
train["연휴포함"] = ((holidays['15_sul'].get('day_start') > train["개봉일"]) 
                 & (holidays['15_sul'].get('day_end') < train["날짜"]))
train["연휴포함"] = ((holidays['15_chu'].get('day_start') > train["개봉일"]) 
                 & (holidays['15_chu'].get('day_end') < train["날짜"]))
train["연휴포함"] = ((holidays['16_sul'].get('day_start') > train["개봉일"]) 
                 & (holidays['16_sul'].get('day_end') < train["날짜"]))

## Actors Feature
- 정규표현식을 통해 외국인 배우 이름 반영

In [9]:
train['배우'] = train['배우'].fillna("")

In [10]:
train['배우'] = train['배우'].apply(lambda name: re.sub('[123456789]', '', name))
train['배우'] = train['배우'].apply(lambda name: str(name).replace(" ", ""))
train['배우'] = train['배우'].apply(lambda name: str(name).replace("-", ""))

test['배우'] = test['배우'].apply(lambda name: re.sub('[123456789]', '', name))
test['배우'] = test['배우'].apply(lambda name: str(name).replace(" ", ""))
test['배우'] = test['배우'].apply(lambda name: str(name).replace("-", ""))

## Info Feature
- 대표국적, 제작사, 배급사, 등급, 장르, 감독, 배우를 통합

In [11]:
vector_columns = test.columns.drop(['날짜', '영화명', '개봉일', '국적'])
vector_columns

Index(['대표국적', '제작사', '배급사', '등급', '장르', '감독', '배우'], dtype='object')

In [12]:
d_list = []
for each in train.iterrows():
    row = ""
    for column in vector_columns.values :
        if type(each[1][column]) == str :
            row = row + each[1][column] + " "
    d_list.append(row)

In [13]:
test_d_list = []
for each in test.iterrows():
    row = ""
    for column in vector_columns.values :
        if type(each[1][column]) == str :
            row = row + each[1][column] + " "
    test_d_list.append(row)

In [14]:
test_d_list

['한국 (주)시네마서비스 씨제이이앤엠(주) 전체관람가 사극 강우석 차승원,유준상,김인권,남지현,신동미,남경읍,태인호,공형진,김종수 ',
 '미국  유니버설픽쳐스인터내셔널 코리아(유) 15세이상관람가 액션, 서부극(웨스턴) 안톤 후쿠아 덴젤워싱턴,크리스프랫,이병헌,에단호크,맷보머,빈센트도노프리오,해일리베넷,캠지갠뎃 ',
 '미국  CGV아트하우스 15세이상관람가 멜로/로맨스, 드라마 우디 앨런 제시아이젠버그,크리스틴스튜어트,스티브카렐,블레이크라이블리 ']

## Vectorize


In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, vstack, hstack

In [16]:
vectorizer = TfidfVectorizer()
x_list = vectorizer.fit_transform(d_list)

In [17]:
days = train['상영기간'].values
hol = train['연휴포함'].values

In [18]:
mat = csr_matrix(days , dtype=int)
mat.transpose()
x_list = hstack((x_list, mat.transpose()))

In [19]:
mat = csr_matrix(hol , dtype=int)
mat.transpose()
x_list = hstack((x_list, mat.transpose()))

# Cross Validation
- TrainSet과 TestSet을 분할
- RMSE(Root Mean Squared Error) 계산
- K-Fold를 통해 의미있는 RMSE 값을 도출

In [20]:
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold

In [21]:
x_list.shape

(7393, 2272)

In [22]:
y_list = train['누적관객수'].values

In [23]:
def RMSE(y, ypred) :
    rmse = np.sqrt(np.mean((y - ypred) **2))
    return rmse

In [24]:
cv = KFold(n=len(y_list), n_folds=5, shuffle=True)
cv.idxs

array([2635, 1576, 4408, ..., 7231, 2019, 5408])

# RMSE
- RandomForestRegressor Model을 사용
- Hyper Parameter Tuning을 통해 최적의 파라메터를 분석

In [25]:
from sklearn.ensemble import RandomForestRegressor

In [26]:
x_array = x_list.toarray()
y_array = y_list
rmse_list = []

In [27]:
for training_set, test_set in cv:
    X_train = x_array[training_set]
    y_train = y_array[training_set]
    X_test = x_array[test_set]
    y_test = y_array[test_set]
    model = RandomForestRegressor(n_estimators=100, n_jobs=-1)
    model.fit(csr_matrix(X_train, dtype=float), y_train)
    y_prediction = model.predict(X_test)
    rmse = RMSE(y_test,y_prediction)
    rmse_list.append(rmse)

In [28]:
print("RMSE = {0:.6f}".format(sum(rmse_list) / len(rmse_list)))

RMSE = 112744.583357


In [29]:
test_x_list = vectorizer.transform(test_d_list)

# Predict
- For test data

In [30]:
from sklearn.ensemble import RandomForestRegressor
model_rfr = RandomForestRegressor(n_estimators=100, n_jobs=-1)
model_rfr.fit(x_list, y_list)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [31]:
test_days = [23,16,16]

In [32]:
mat = csr_matrix(test_days , dtype=int)
mat.transpose()
test_x_list = hstack((test_x_list, mat.transpose()))

In [33]:
hol = csr_matrix([True,True,True] , dtype=int)
hol.transpose()
test_x_list = hstack((test_x_list, hol.transpose()))

In [34]:
model_rfr.predict(test_x_list)

array([ 1468606.16,  1766153.99,   514653.49])

# Result


## Last Baseline

- RMSE = 112744.583357
- Predict = 1468606.16,  1766153.99,   514653.49